In [1]:
# plotly standard imports
import plotly.graph_objs as go
import chart_studio.plotly as py

# Cufflinks wrapper on plotly
import cufflinks

# Data science imports
import pandas as pd
import numpy as np

# Options for pandas
pd.options.display.max_columns = 30

# Display all cell outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [2]:

from plotly.offline import iplot, init_notebook_mode
cufflinks.go_offline(connected=True)
init_notebook_mode(connected=True)

# Set global theme
cufflinks.set_config_file(world_readable=True, theme='pearl')

## Lets explore datasets

In [22]:
import tensorflow_datasets as tfds
import src

train_data, validation_data, test_data = src.download()

train_df = pd.DataFrame(tfds.as_numpy(train_data), columns=['text', 'type'])

label_categories = ['bad', 'good']

train_df['type'] = train_df['type'].apply(lambda x: label_categories[x])

train_df.head()

,text,type
0,"b""This was an absolutely terrible movie. Don't...",bad
1,b'I have been known to fall asleep during film...,bad
2,b'Mann photographs the Alberta Rocky Mountains...,bad
3,b'This is the kind of film for a snowy Sunday ...,good
4,"b'As others have mentioned, all the women that...",good


### Training data distribution

In [23]:
train_df['type'].iplot(
    kind='hist',
    yTitle='count',
    xTitle='Type',
    title='Training data distribution'
)

## Explore training metrics

In [12]:
df = pd.read_csv('./metrics/training.csv')
df.head()

,epoch,accuracy,loss,val_accuracy,val_loss
0,0,0.72425,0.581042,0.8064,0.471475
1,1,0.80370,0.448358,0.8276,0.402654
2,2,0.82610,0.402184,0.8354,0.377996
3,3,0.83375,0.384337,0.8384,0.370425
4,4,0.83880,0.373361,0.8446,0.361078


In [13]:
df[['epoch', 'accuracy', 'val_accuracy']].iplot(
    x='epoch',
    mode='lines+markers',
    xTitle='epoch',
    yTitle='accuracy', 
    title='Training accuracy',
    linecolor='black',
)

In [14]:
df[['epoch', 'loss', 'val_loss']].iplot(
    x='epoch',
    mode='lines+markers',
    xTitle='epoch',
    yTitle='accuracy', 
    title='Losses'
)

## Predictions

### Load probability model

which can give predictions on model classes

0 - bad review, 1 - good revie

In [15]:

model = src.get_probability_model()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               38528     
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 258       
Total params: 38,786
Trainable params: 38,786
Non-trainable params: 0
_________________________________________________________________


**Firstly will try predict on some data from training dataset**

In [24]:
REVIEW_INDEX = 110

text, real_label = src.get_text_and_label_from_dataset(REVIEW_INDEX)

print('text for prediction\n\n', text, '\n')

predicted_label, predictions = src.predict(text, model)

print(label_categories[predicted_label], 'review')

print('\n\nPredicted label:', predicted_label, 'real label: ', real_label, 'predictions:', predictions)
if (predicted_label == real_label):
    print('Successfully predicted')
else:
    print('Failed to predict')

text for prediction

 Dakota (1988) was another early Lou Diamond Phillips starring vehicle. This film is similar to the later released film Harley. There are a few differences but they're both the same. I don't know which one came first. I guess it'll remain one of the mysteries of life. But they both are troubled "kids" who are trying to turn there lives around. Instead of bikes this one involves horses. They're basically the same movie and they're both cheesy as hell. If you're a serious L.D.P. fan then I recommend that you watch them both. You get some extreme mugging and posturing from L.D.P. if you're game then go for it.<br /><br />Not recommended, except for L.D.P. fans!!! 

good review


Predicted label: 1 real label:  0 predictions: [0.15473373 0.8452662 ]
Failed to predict


**Then will try predict hadnwritten text**

In [25]:
# Can change text and check model
hadwriten = 'This is good film'

print('Hendwriten text:\n', hadwriten, '\n')

handwriten_label, predictions = src.predict(hadwriten, model)

print(label_categories[predicted_label], 'review')

print('Probabilities', predictions)


Hendwriten text:
 This is good film 

good review
Probabilities [0.00197127 0.9980287 ]
